In [ ]:
import os
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install -q google-genai google-auth python-dotenv numpy
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
else:
    def find_service_account_json(max_up=6):
        p = Path.cwd()
        for _ in range(max_up):
            candidate = p / "series-2-coding-llms" / "creds"
            if candidate.exists():
                for f in candidate.glob("*.json"):
                    return str(f.resolve())
            candidate2 = p / "creds"
            if candidate2.exists():
                for f in candidate2.glob("*.json"):
                    return str(f.resolve())
            p = p.parent
        return None

    sa_path = find_service_account_json()
    if sa_path:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    else:
        try:
            from dotenv import load_dotenv
            load_dotenv()
        except Exception:
            pass

import google.auth
from google import genai
from google.genai import types

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
client = genai.Client(vertexai=True, project=project, location="us-central1")
print(f"Using project: {project}")

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kgweber-cwru/coding-with-ai-wn26/blob/main/week-5-building-rag-systems/assignment.ipynb)

# Troubleshooting: Colab, .env, and GitHub 404s

- **Colab ignores local `.env` files.** Use the prompt at the top of the notebook to set your `GOOGLE_CLOUD_PROJECT`, or run `os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"` in a cell.
- **If you see a `404 Not Found` when opening via the "Open in Colab" badge:** this usually means the badge is pointing to a branch/path that doesn't contain the notebook (branch mismatch). Fix options:
  - Push the notebooks to the branch referenced by the badge (commonly `main`).
  - Or update the badge URL to the repository branch that has the notebooks.
- **If Colab prompts for GitHub access or fails to load:** sign in to Colab and authorize GitHub access, then retry using the badge or use `File → Open notebook → GitHub` and search the repo.
- **Workarounds:** if issues persist, use `File → Save a copy in Drive` from the Colab error view or open the notebook directly from GitHub and copy into Colab.
- **Transient errors:** GitHub API rate limits or transient failures can cause temporary 404s — wait a minute and retry if everything else looks correct.

# Week 5 Assignment: Build a Domain-Specific RAG System

## Objective
Build a complete, production-quality RAG system for your domain with proper chunking, metadata, and retrieval strategies.

## Requirements
1. Use the RAGSystem class (or build your own)
2. Process at least 15 documents with metadata
3. Implement appropriate chunking
4. Test with 5+ realistic questions
5. Demonstrate metadata filtering
6. Evaluate retrieval quality

In [ ]:
import os
import numpy as np
import json
from datetime import datetime
from dotenv import load_dotenv
from google import genai
from google.genai import types
import google.auth

load_dotenv('../.env')
creds, project = google.auth.default()
client = genai.Client(vertexai=True, project=project, location="us-central1")

## Copy Classes from Concepts Notebook

(Include Document, DocumentProcessor, RAGDocumentStore, and RAGSystem classes)

In [ ]:
# COPY CLASSES HERE

## Step 1: Describe Your System

**YOUR DESCRIPTION**

Domain: [your field]

Use case: [what problem does this solve?]

Document types: [what you're indexing]

Target users: [who will use this?]

Metadata strategy: [what metadata will you track and why?]

## Step 2: Prepare Your Documents

In [ ]:
# Create your document collection with metadata
documents = [
    {
        "content": """YOUR DOCUMENT CONTENT""",
        "metadata": {
            "source": "...",
            "category": "...",
            # Add relevant metadata
        }
    },
    # Add at least 15 documents
]

print(f"Prepared {len(documents)} documents")

## Step 3: Build Your RAG System

In [ ]:
# Initialize system
rag = RAGSystem()

# Add documents
print("Building knowledge base...")
for doc in documents:
    chunks = rag.add_document(
        doc["content"],
        doc["metadata"],
        chunk_strategy='sentences',  # or 'words'
        chunk_size=3  # adjust as needed
    )
    print(f"  Added document: {chunks} chunks")

print(f"\nSystem ready! Stats: {rag.stats()}")

## Step 4: Test with Questions

In [ ]:
test_questions = [
    "YOUR QUESTION 1",
    "YOUR QUESTION 2",
    "YOUR QUESTION 3",
    "YOUR QUESTION 4",
    "YOUR QUESTION 5",
]

for q in test_questions:
    print(f"\n{'='*70}")
    print(f"Q: {q}")
    print('='*70)
    
    result = rag.query(q, top_k=3)
    
    print(f"\nA: {result['answer']}")
    print("\nSources:")
    for i, src in enumerate(result['sources'], 1):
        print(f"  {i}. [Score: {src['similarity']:.3f}]")
        print(f"     {src['chunk'][:100]}...")
        print(f"     Metadata: {src['metadata']}")

## Step 5: Demonstrate Filtering

In [ ]:
# Test metadata filtering
filtered_question = "YOUR QUESTION"
filter_criteria = {"YOUR_METADATA_KEY": "YOUR_VALUE"}

print(f"Question: {filtered_question}")
print(f"Filter: {filter_criteria}\n")

result = rag.query(filtered_question, filters=filter_criteria, top_k=3)
print(f"Answer: {result['answer']}")

## Step 6: Evaluate Retrieval Quality

In [ ]:
# Manually evaluate: for each question, were the right chunks retrieved?

evaluation = [
    {
        "question": "YOUR QUESTION",
        "relevant_chunks_found": True/False,
        "answer_quality": "good/fair/poor",
        "notes": "YOUR OBSERVATIONS"
    },
    # Evaluate each test question
]

# Calculate metrics
success_rate = sum(1 for e in evaluation if e['relevant_chunks_found']) / len(evaluation)
print(f"Retrieval success rate: {success_rate:.1%}")

## Reflection

### 1. How did chunking affect retrieval quality?

**YOUR ANSWER**

### 2. Was metadata filtering useful? When would you use it?

**YOUR ANSWER**

### 3. What improvements would make this production-ready?

**YOUR ANSWER**

### 4. What challenges did you encounter?

**YOUR ANSWER**

## Bonus: Save/Load System

Implement persistence so your system can be saved and loaded:

In [ ]:
# BONUS CODE HERE